# Evidently LLM tracing tutorial

In [ ]:
# ! pip install evidently

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 920.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.5/531.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.

In [1]:
# ! pip install tracely

  Obtaining dependency information for tracely from https://files.pythonhosted.org/packages/c6/e4/ae9f88a8fb4e81889d080fcf606fdc7c123bd6185b66f29b2b10b56117c2/tracely-0.1.0-py3-none-any.whl.metadata


In [ ]:
# !pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
from io import BytesIO
import openai

In [3]:
from tracely import init_tracing
from tracely import trace_event

## Initializatin

Evidently Cloud: create an Account, Organization, Team and get an API key: https://app.evidently.cloud/token

OpenAI API key: https://platform.openai.com/api-keys

In [ ]:
openai_api_key = "YOUR_OPEN_AI_API_KEY" # os.environ["OPENAI_API_KEY"] = "YOUR_KEY"
my_token = "YOUR_EVIDENTLY_TOKEN"

In [ ]:
address="https://app.evidently.cloud/"
team_id="YOUR_TEAM_ID"
dataset_name="YOUR_TRACING_DATASET_NAME"

Use ```init_tracing``` to enable tracely tracing.

**Signature:** init_tracing(address: Optional[str] = None, exporter_type: Optional[str] = None, api_key: Optional[str] = None, team_id: Optional[str] = None, export_name: Optional[str] = None, *, as_global: bool = True) -> opentelemetry.trace.TracerProvider

**Parameters:**
 - **address:** address of collector service
 - **exporter_type:** type of exporter to use "grpc" or "http"
 - **api_key:** authorization API key for Evidently tracing
 - **team_id:** team ID in Evidently Cloud
 * **export_name**: string name of exported data, all data with the same ID would be grouped into single dataset
 - **as_global:** indicated when to register provider globally for opentelemetry or use local one

In [ ]:
init_tracing(
    address=address,
    api_key=my_token,
    team_id=team_id,
    export_name=dataset_name
    )

## Tracing simple example

In [ ]:
client = openai.OpenAI(api_key=openai_api_key)

In [ ]:
question_list = [
    "What are the main differences between climate change and global warming?",
    "What are the key principles of quantum mechanics?",
    "Can you explain the theory of relativity in simple terms?",
    "How do vaccines work?",
    "What is blockchain technology and how does it work?",
    "What are the potential benefits and risks of artificial intelligence?",
    "What are the distinguishing features of Baroque art?",
    "What are some effective ways to manage stress?",
    "What is the difference between stocks and bonds?",
    "Can you explain the concept of cryptocurrency and how it works?",

]

In [ ]:
question_prompt = """
Please answer the following question nicely with a clear structure of response and some conclusions at the end.

Here is a question: {{text_input}}
"""

To trace a function call use ```trace_event()``` decorator

**Signature:**

trace_event(span_name: Optional[str] = None, track_args: Optional[List[str]] = None, ignore_args: Optional[List[str]] = None, track_output: Optional[bool] = True)

**Parameters:**
  *  **span_name:** the name of the span to track as.
  * **track_args:** list of arguments to capture, if set to None - capture all arguments (default), if set to [] do not capture any arguments
  * **ignore_args:** list of arguments to ignore, if set to None - do not ignore any arguments.
  * **track_output:** track the output of the function call

**Common cases:**
- ```@trace_event()``` - log all arguments of the function
- ```@trace_event(track_args=[])``` - log only input arguments of the function
- ```@trace_event(track_args=["arg1", "arg2"])``` - log only "arg1" and "arg2"

In [ ]:
@trace_event()
def pseudo_assistant(prompt, question):
  model = "gpt-4o-mini"
  system_prompt = "You are a nice and helpful assistant "
  user_prompt = prompt.replace("{{text_input}}", question)

  choices = client.chat.completions.create(
    model=model,
    messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_prompt},
    ]
  )

  response = choices.choices[0].message.content

  return response

In [ ]:
for question in question_list:
   pseudo_assistant(prompt=question_prompt, question=question)

## Load traced data

In [ ]:
from evidently.ui.workspace.cloud import CloudWorkspace

from evidently.report import Report

from evidently import metrics
from evidently.metric_preset import DataQualityPreset

In [ ]:
ws = CloudWorkspace(token=my_token, url=address)

In [ ]:
dataset_id = "YOUR_DATASET_ID"

In [ ]:
traced_data = ws.load_dataset(dataset_id = dataset_id)
traced_data.head()

,id,service_name,timestamp,pseudo_assistant.prompt,pseudo_assistant.question,pseudo_assistant.result,pseudo_assistant.exception
0,032f4bb7-b776-2e81-0d48-22081f686817,unknown_service,2024-08-06 14:53:09.968997,\nPlease answer the following question nicely ...,What are the distinguishing features of Baroqu...,"Baroque art, which flourished from the late 16...",None
1,0888f342-7ef6-3561-6600-b3d7f4476378,unknown_service,2024-08-06 14:52:36.765640,\nPlease answer the following question nicely ...,How do vaccines work?,Vaccines are an essential tool in public healt...,None
2,0a390bc5-7d1b-3c68-5b75-36724252fe9a,unknown_service,2024-08-06 11:34:06.701821,\nPlease answer the following question nicely ...,What are the main differences between climate ...,Certainly! Let’s explore the key differences b...,None
3,0dc7bed7-4f53-5f57-f411-b0b29a169626,unknown_service,2024-08-06 14:52:56.647948,\nPlease answer the following question nicely ...,What are the potential benefits and risks of a...,**Question: What are the potential benefits an...,None
4,24406289-b034-5075-b4d0-b185330c3704,unknown_service,2024-08-06 14:52:25.939959,\nPlease answer the following question nicely ...,Can you explain the theory of relativity in si...,"Certainly! The theory of relativity, proposed ...",None
